### Yolo v5 다운로드 및 설치

In [1]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5;pip install -qr requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 15973, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 15973 (delta 87), reused 113 (delta 75), pack-reused 15831
Receiving objects: 100% (15973/15973), 14.60 MiB | 7.44 MiB/s, done.
Resolving deltas: 100% (10956/10956), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.4/595.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [2]:
# Google Drive 접근을 위한 Mount 적용. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# soft link로 Google Drive Directory 연결. 
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# Google Drive 밑에 Directory 생성. 이미 생성 되어 있을 시 오류 발생. 
!mkdir "/mydrive/ultra_workdir"

'Backup 2023-04-13.zip'   cropped_images   raw_images	  ultra_workdir
'Colab Notebooks'	  LP_data	   sample_image  '제목 없는 문서.gdoc'
mkdir: cannot create directory ‘/mydrive/ultra_workdir’: File exists


### Dataset용 yaml 파일을 생성하고 학습 수행
* yolo v5는 모델이 yolov5s(small), yolov5m(middle), yolov5l(large), yolov5x(extra large)로 되어있음. weight 인자값으로 이들중 하나를 입력해 줌


In [ ]:
### batch size는 8로 정할것. 16으로 설정시 성능이 좋아지지 않음. epoch는 30번으로 설정.
### 번호판 검출의 경우 상대적으로 mAP:0.5~0.95 Detection 성능이 좋지 못함. 
### 학습데이터가 적은것도 이유지만, Object 사이즈가 상대적으로 작아서 그런듯 함. 
!cd /content/yolov5; python train.py --img 640 --batch 8 --epochs 150 --data /content/gdrive/MyDrive/LP_data/LP_data.yaml --weights yolov5l.pt \
                                     --project=/mydrive/ultra_workdir --name LP --exist-ok 

### 단일 검출 모델(raw_images)에 대한 inference 수행

In [4]:
# raw_image 파일 inference 
!cd /content/yolov5; python val.py --weights /mydrive/ultra_workdir/LP/weights/best.pt  \
                           --data /content/gdrive/MyDrive/LP_data/LP_data.yaml \
                           --project /content/data/output --name=raw_test_result --exist-ok --img 640 --iou 0.65

val: data=/content/gdrive/MyDrive/LP_data/LP_data.yaml, weights=['/mydrive/ultra_workdir/LP/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/data/output, name=raw_test_result, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v7.0-177-g89c3040 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
100% 755k/755k [00:00<00:00, 108MB/s]
val: Scanning /content/gdrive/MyDrive/LP_data/val/labels... 0 images, 38 backgrounds, 0 corrupt:  64% 38/59 [00:26<00:14,  1.43it/s]
Traceback (most recent call last):
  File "/content/yolov5/utils/dataloaders.py", line 491, in __init__
    assert cache['hash'] == get_hash(self.label_files + self.im_files)  # identical hash
AssertionError

During handling of 

### 이중 검출 모델(car_images)에 대한 inference 수행 

In [ ]:
# car_image 파일 inference 
!cd /content/yolov5; python val.py --weights /mydrive/ultra_workdir/LP/weights/best.pt  \
                           --data /content/gdrive/MyDrive/LP_data/LP_data.yaml \
                           --project /content/data/output --name=car_test_result --exist-ok --img 640 --iou 0.65

### 단일 검출 모델(raw_images)에 대한 실사용

In [10]:
!cd /content/yolov5;python detect.py --source /content/gdrive/MyDrive/LP_data/val/images \
                            --weights /mydrive/ultra_workdir/LP/weights/best.pt  --conf 0.3 \
                            --project=/content/data/output --name=run_image --exist-ok --line-thickness 2

detect: weights=['/mydrive/ultra_workdir/LP/weights/best.pt'], source=/content/gdrive/MyDrive/LP_data/val/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/data/output, name=run_image, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-177-g89c3040 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
image 1/29 /content/gdrive/MyDrive/LP_data/val/images/LP_F (100).JPEG: 384x640 3 Llicense_Plates, 45.3ms
image 2/29 /content/gdrive/MyDrive/LP_data/val/images/LP_F (104).JPEG: 384x640 4 Llicense_Plates, 32.1ms
image 3/29 /content/gdrive/MyDrive/LP_data/val/images/LP_F (108).JPEG: 384x640 3 Llicen

### 이중 검출 모델(car_images)에 대한 실사용

In [11]:
!cd /content/yolov5;python detect.py --source /content/gdrive/MyDrive/LP_data/double/images \
                            --weights /mydrive/ultra_workdir/LP/weights/best.pt  --conf 0.3 \
                            --project=/content/data/output --name=run_image4 --exist-ok --line-thickness 2

detect: weights=['/mydrive/ultra_workdir/LP/weights/best.pt'], source=/content/gdrive/MyDrive/LP_data/double/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/data/output, name=run_image4, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-177-g89c3040 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
image 1/4 /content/gdrive/MyDrive/LP_data/double/images/a.JPG: 512x640 (no detections), 49.3ms
image 2/4 /content/gdrive/MyDrive/LP_data/double/images/b.JPG: 480x640 (no detections), 48.7ms
image 3/4 /content/gdrive/MyDrive/LP_data/double/images/c.JPG: 512x640 (no detections), 40.2ms
image 4/

### video inference 수행(real time)


In [ ]:
# !cd /content/yolov3;python detect.py --source /content/incredibles/test.mp4 \
#                             --weights /mydrive/ultra_workdir/incredibles/weights/best.pt --conf 0.3 \
#                             --project=/content/data/output --name=run_image --exist-ok --line-thickness 1